In [ ]:
import DBcm
import swim_utils
import os
import datetime

In [ ]:
config = {
    "user": "swimuser",
    "password": "swimuserpasswd",
    "database": "SwimclubDB",
    "host": "localhost",
}

In [ ]:
PROCESSING_FOLDER = "ToUploadSwimData"
UPLOADED_FOLDER = "UploadedSwimData"

In [ ]:
def query_swimmers() -> list:
    SQL: str = """SELECT * from swimmers"""

    with DBcm.UseDatabase(config) as db:
        db.execute(SQL)
        data: list = db.fetchall()

    return [row[1] + "-" + str(row[2]) for row in data]

In [ ]:
def query_events() -> list:
    SQL: str = """SELECT * FROM events"""

    with DBcm.UseDatabase(config) as db:
        db.execute(SQL)
        data: list = db.fetchall()

    return [row[1] + "-" + row[2] for row in data]

In [ ]:
def upload_file_data(file_name: str) -> None:
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file_name)

    swimmers: list = query_swimmers()
    events: list = query_events()

    with DBcm.UseDatabase(config) as db:
        if name + "-" + age not in swimmers:
            SQL: str = f"""INSERT INTO swimmers (name, age) VALUES ("{name}", {age})"""
            db.execute(SQL)

        SQL: str = (
            f"""SELECT swimmer_id FROM swimmers WHERE name="{name}" AND age={age} """
        )
        db.execute(SQL)
        swimmer_id: int = db.fetchall()[0][0]

        if distance + "-" + stroke not in events:
            SQL: str = f"""INSERT INTO events (distance, stroke) VALUES ("{distance}", "{stroke}")"""
            db.execute(SQL)

        SQL: str = f"""SELECT event_id FROM events WHERE distance="{distance}" AND stroke="{stroke}" """
        db.execute(SQL)
        event_id: int = db.fetchall()[0][0]

        for time in the_times:
            SQL: str = f"""INSERT INTO times (swimmer_id, event_id, time, ts) VALUES ({swimmer_id}, {event_id}, "0:{time}", "{timestamp}")"""
            db.execute(SQL)

In [ ]:
if PROCESSING_FOLDER not in os.listdir():
    os.makedirs(PROCESSING_FOLDER)
print(f"Put new folder with files in {PROCESSING_FOLDER}.")

if UPLOADED_FOLDER not in os.listdir():
    os.makedirs(UPLOADED_FOLDER)
print(f"Files uploaded are moved to {UPLOADED_FOLDER}.")

In [ ]:
directories: list = os.listdir(PROCESSING_FOLDER)
if len(directories) == 0:
    print(
        f"No folders found. Make sure files are in a folder and then in {PROCESSING_FOLDER}."
    )
else:
    files: list = os.listdir(PROCESSING_FOLDER + "/" + directories[0])
    if len(files) == 0:
        print(
            f"No files found. Make sure files are in a folder and then in {PROCESSING_FOLDER}."
        )
    else:
        if directories[0] not in os.listdir(UPLOADED_FOLDER):
            os.makedirs(UPLOADED_FOLDER + "/" + directories[0])
        
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        for file in files:
            upload_file_data(PROCESSING_FOLDER + "/" + directories[0] + "/" + file)
            os.rename(
                PROCESSING_FOLDER + "/" + directories[0] + "/" + file,
                UPLOADED_FOLDER + "/" + directories[0] + "/" + file,
            )
        os.rmdir(PROCESSING_FOLDER + "/" + directories[0])
        print("Files uploaded.")